<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/ac-willeke/urban-climate"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" style="filter: invert(100%)"/> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://colab.research.google.com/github/NINAnor/mapper-soilCondition/blob/main/src/data/GEE_display_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Caclulate statistics per district using DuckDB

**Author**: Willeke A'Campo

**Description:** This notebooks shows how to calculate the Ecosystem Service statistics per district using DuckDB. The results are stored in a new table in the database and exported to GeoJSON.

**Documentation:** 